In [3]:
%cd /Users/akurzela/projects/ashrae_energy_prediction

/Users/akurzela/projects/ashrae_energy_prediction


In [4]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
from src.data import get_weather_data, get_building_data, get_base_data
import seaborn as sns
import numpy as np
pd.set_option('display.max_columns', 500)
from sklearn.ensemble import RandomForestRegressor
from sklearn.datasets import make_regression

df_weather = get_weather_data()
df_building = get_building_data()
df_base = get_base_data()

df_base_building = pd.merge(df_base, df_building, how="left", on=["building_id"])
df = pd.merge(
    df_base_building, df_weather, how="left", on=["site_id", "timestamp"]
)


In [3]:
df.head()

,building_id,meter,timestamp,meter_reading,row_id,site_id,primary_use,square_feet,year_built,floor_count,air_temperature,cloud_coverage,dew_temperature,precip_depth_1_hr,sea_level_pressure,wind_direction,wind_speed
0,0,0,2016-01-01 00:00:00,0.0,NaN,0,Education,7432,2008.0,NaN,25.0,6.0,20.0,NaN,1019.7,0.0,0.0
1,1,0,2016-01-01 00:00:00,0.0,NaN,0,Education,2720,2004.0,NaN,25.0,6.0,20.0,NaN,1019.7,0.0,0.0
2,2,0,2016-01-01 00:00:00,0.0,NaN,0,Education,5376,1991.0,NaN,25.0,6.0,20.0,NaN,1019.7,0.0,0.0
3,3,0,2016-01-01 00:00:00,0.0,NaN,0,Education,23685,2002.0,NaN,25.0,6.0,20.0,NaN,1019.7,0.0,0.0
4,4,0,2016-01-01 00:00:00,0.0,NaN,0,Education,116607,1975.0,NaN,25.0,6.0,20.0,NaN,1019.7,0.0,0.0


## I am creating dummy variables for primary_use

In [5]:
df = pd.get_dummies(df, columns=['primary_use'])

In [6]:
df = pd.get_dummies(df, columns=['meter'])

In [6]:
df.head()

,building_id,timestamp,meter_reading,row_id,site_id,square_feet,year_built,floor_count,air_temperature,cloud_coverage,dew_temperature,precip_depth_1_hr,sea_level_pressure,wind_direction,wind_speed,primary_use_Education,primary_use_Entertainment/public assembly,primary_use_Food sales and service,primary_use_Healthcare,primary_use_Lodging/residential,primary_use_Manufacturing/industrial,primary_use_Office,primary_use_Other,primary_use_Parking,primary_use_Public services,primary_use_Religious worship,primary_use_Retail,primary_use_Services,primary_use_Technology/science,primary_use_Utility,primary_use_Warehouse/storage,meter_0,meter_1,meter_2,meter_3
0,0,2016-01-01 00:00:00,0.0,NaN,0,7432,2008.0,NaN,25.0,6.0,20.0,NaN,1019.7,0.0,0.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
1,1,2016-01-01 00:00:00,0.0,NaN,0,2720,2004.0,NaN,25.0,6.0,20.0,NaN,1019.7,0.0,0.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
2,2,2016-01-01 00:00:00,0.0,NaN,0,5376,1991.0,NaN,25.0,6.0,20.0,NaN,1019.7,0.0,0.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
3,3,2016-01-01 00:00:00,0.0,NaN,0,23685,2002.0,NaN,25.0,6.0,20.0,NaN,1019.7,0.0,0.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
4,4,2016-01-01 00:00:00,0.0,NaN,0,116607,1975.0,NaN,25.0,6.0,20.0,NaN,1019.7,0.0,0.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0


## I am filling NAs

In [7]:
df = df.fillna(-100)

## I am checking correlation of the values to see which ones I can remove and create new ones

In [8]:
corr = df[['wind_speed', 'wind_direction']].corr()
print(corr)

                wind_speed  wind_direction
wind_speed        1.000000        0.265442
wind_direction    0.265442        1.000000


In [9]:
df['wind_direction'].unique()

array([   0.,  240.,  270.,  350., -100.,  280.,  200.,  250.,   70.,
        230.,  360.,  260.,   20.,  190.,  300.,  330.,   10.,   50.,
        170.,   90.,  160.,   30.,  340.,  110.,  310.,  130.,  290.,
        100.,  120.,   60.,  150.,  210.,   80.,   40.,  320.,  140.,
        220.,  180.,  174.,  176.,   84.,  198.,  196.,  202.,  359.,
        331.,  319.,  288.,   77.,  123.,  222.,  175.,  185.,  217.,
        201.,  255.,   59.,  211.,  213.,   26.,  284.,  129.,  239.,
        262.,  275.,  302.,  264.])

In [8]:
wind_ratio = df['wind_speed']/df['wind_direction']

In [9]:
df['wind_ratio']=wind_ratio

In [10]:
temperature_ratio = df['air_temperature']/df['dew_temperature']

In [11]:
df['temperature_ratio'] = temperature_ratio

In [14]:
df.head()

,building_id,timestamp,meter_reading,row_id,site_id,square_feet,year_built,floor_count,air_temperature,cloud_coverage,dew_temperature,precip_depth_1_hr,sea_level_pressure,wind_direction,wind_speed,primary_use_Education,primary_use_Entertainment/public assembly,primary_use_Food sales and service,primary_use_Healthcare,primary_use_Lodging/residential,primary_use_Manufacturing/industrial,primary_use_Office,primary_use_Other,primary_use_Parking,primary_use_Public services,primary_use_Religious worship,primary_use_Retail,primary_use_Services,primary_use_Technology/science,primary_use_Utility,primary_use_Warehouse/storage,meter_0,meter_1,meter_2,meter_3,wind_ratio,names_ratio
0,0,2016-01-01 00:00:00,0.0,-100.0,0,7432,2008.0,-100.0,25.0,6.0,20.0,-100.0,1019.7,0.0,0.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,NaN,1.25
1,1,2016-01-01 00:00:00,0.0,-100.0,0,2720,2004.0,-100.0,25.0,6.0,20.0,-100.0,1019.7,0.0,0.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,NaN,1.25
2,2,2016-01-01 00:00:00,0.0,-100.0,0,5376,1991.0,-100.0,25.0,6.0,20.0,-100.0,1019.7,0.0,0.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,NaN,1.25
3,3,2016-01-01 00:00:00,0.0,-100.0,0,23685,2002.0,-100.0,25.0,6.0,20.0,-100.0,1019.7,0.0,0.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,NaN,1.25
4,4,2016-01-01 00:00:00,0.0,-100.0,0,116607,1975.0,-100.0,25.0,6.0,20.0,-100.0,1019.7,0.0,0.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,NaN,1.25


## Getting rid of correlated variables and replacing them with ratios

In [12]:
df.drop(columns=['air_temperature','dew_temperature', 'row_id', 'wind_direction', 'wind_speed'], axis =1)

,building_id,timestamp,meter_reading,site_id,square_feet,year_built,floor_count,cloud_coverage,precip_depth_1_hr,sea_level_pressure,primary_use_Education,primary_use_Entertainment/public assembly,primary_use_Food sales and service,primary_use_Healthcare,primary_use_Lodging/residential,primary_use_Manufacturing/industrial,primary_use_Office,primary_use_Other,primary_use_Parking,primary_use_Public services,primary_use_Religious worship,primary_use_Retail,primary_use_Services,primary_use_Technology/science,primary_use_Utility,primary_use_Warehouse/storage,meter_0,meter_1,meter_2,meter_3,wind_ratio,temperature_ratio
0,0,2016-01-01 00:00:00,0.0,0,7432,2008.0,-100.0,6.0,-100.0,1019.7,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,NaN,1.25
1,1,2016-01-01 00:00:00,0.0,0,2720,2004.0,-100.0,6.0,-100.0,1019.7,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,NaN,1.25
2,2,2016-01-01 00:00:00,0.0,0,5376,1991.0,-100.0,6.0,-100.0,1019.7,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,NaN,1.25
3,3,2016-01-01 00:00:00,0.0,0,23685,2002.0,-100.0,6.0,-100.0,1019.7,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,NaN,1.25
4,4,2016-01-01 00:00:00,0.0,0,116607,1975.0,-100.0,6.0,-100.0,1019.7,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,NaN,1.25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61913695,1444,2018-05-09 07:00:00,-100.0,15,19619,1914.0,-100.0,-100.0,-100.0,-100.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1.0,1.00
61913696,1445,2018-05-09 07:00:00,-100.0,15,4298,-100.0,-100.0,-100.0,-100.0,-100.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1.0,1.00
61913697,1446,2018-05-09 07:00:00,-100.0,15,11265,1997.0,-100.0,-100.0,-100.0,-100.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1.0,1.00
61913698,1447,2018-05-09 07:00:00,-100.0,15,29775,2001.0,-100.0,-100.0,-100.0,-100.0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1.0,1.00


In [12]:
df['timestamp'] = pd.to_datetime(df['timestamp'], format='%Y-%m-%d %H:%M:%S')
print(df['timestamp'])

0          2016-01-01 00:00:00
1          2016-01-01 00:00:00
2          2016-01-01 00:00:00
3          2016-01-01 00:00:00
4          2016-01-01 00:00:00
                   ...        
61913695   2018-05-09 07:00:00
61913696   2018-05-09 07:00:00
61913697   2018-05-09 07:00:00
61913698   2018-05-09 07:00:00
61913699   2018-05-09 07:00:00
Name: timestamp, Length: 61913700, dtype: datetime64[ns]


In [13]:
date_year = df['timestamp'].dt.year

In [14]:
df['date_year'] = date_year

In [15]:
age = df['date_year']-df['year_built']

In [16]:
df['age'] = age

In [21]:
df.head()

,building_id,timestamp,meter_reading,row_id,site_id,square_feet,year_built,floor_count,air_temperature,cloud_coverage,dew_temperature,precip_depth_1_hr,sea_level_pressure,wind_direction,wind_speed,primary_use_Education,primary_use_Entertainment/public assembly,primary_use_Food sales and service,primary_use_Healthcare,primary_use_Lodging/residential,primary_use_Manufacturing/industrial,primary_use_Office,primary_use_Other,primary_use_Parking,primary_use_Public services,primary_use_Religious worship,primary_use_Retail,primary_use_Services,primary_use_Technology/science,primary_use_Utility,primary_use_Warehouse/storage,meter_0,meter_1,meter_2,meter_3,wind_ratio,names_ratio,date_year,age
0,0,2016-01-01,0.0,-100.0,0,7432,2008.0,-100.0,25.0,6.0,20.0,-100.0,1019.7,0.0,0.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,NaN,1.25,2016,8.0
1,1,2016-01-01,0.0,-100.0,0,2720,2004.0,-100.0,25.0,6.0,20.0,-100.0,1019.7,0.0,0.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,NaN,1.25,2016,12.0
2,2,2016-01-01,0.0,-100.0,0,5376,1991.0,-100.0,25.0,6.0,20.0,-100.0,1019.7,0.0,0.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,NaN,1.25,2016,25.0
3,3,2016-01-01,0.0,-100.0,0,23685,2002.0,-100.0,25.0,6.0,20.0,-100.0,1019.7,0.0,0.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,NaN,1.25,2016,14.0
4,4,2016-01-01,0.0,-100.0,0,116607,1975.0,-100.0,25.0,6.0,20.0,-100.0,1019.7,0.0,0.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,NaN,1.25,2016,41.0


## I saw NA!

In [17]:
df = df.fillna(-100)

## checking if age is correlated with the meter reading

In [23]:
corr = df[['meter_reading', 'age']].corr()
print(corr)

               meter_reading       age
meter_reading       1.000000  0.005637
age                 0.005637  1.000000


## building a model

In [18]:
df = df.loc[lambda d:d.meter_reading.notnull()] 

## putting logarythm into the target variable

In [19]:
df['meter_reading'] = np.log1p(df['meter_reading'])

/Users/akurzela/projects/ashrae_energy_prediction/.venv/lib/python3.7/site-packages/pandas/core/series.py:856: RuntimeWarning: invalid value encountered in log1p
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [20]:
df['timestamp'].min()

Timestamp('2016-01-01 00:00:00')

## Splitting train and test

In [21]:
df_train = df.loc[df['timestamp'] < '2016-04-01']

In [24]:
df_validation = df.loc[(df['timestamp']>= '2016-04-01') & (df['timestamp']< '2016-09-01')]

In [46]:
df_validation = df_validation.drop(['row_id','meter_reading', 'timestamp', 'year_built'], axis = 1)

In [49]:
df_validation['temperature_ratio'] = df_validation['temperature_ratio'].replace( np.inf, 0)

## selecting variables for the model

In [25]:
target = 'meter_reading'

In [26]:
features = df_train.columns[~df_train.columns.isin(['row_id','meter_reading', 'timestamp', 'year_built'])]

In [39]:
train_features = df_train.drop(['row_id','meter_reading', 'timestamp', 'year_built'], axis = 1)

In [40]:
train_features.max()

building_id                                    1448.00
site_id                                          15.00
square_feet                                  875000.00
floor_count                                      26.00
air_temperature                                  32.80
cloud_coverage                                    9.00
dew_temperature                                  22.80
precip_depth_1_hr                               343.00
sea_level_pressure                             1042.60
wind_direction                                  360.00
wind_speed                                       19.00
primary_use_Education                             1.00
primary_use_Entertainment/public assembly         1.00
primary_use_Food sales and service                1.00
primary_use_Healthcare                            1.00
primary_use_Lodging/residential                   1.00
primary_use_Manufacturing/industrial              1.00
primary_use_Office                                1.00
primary_us

In [36]:
df_train['temperature_ratio'] = df_train['temperature_ratio'].replace( np.inf, 0)

/Users/akurzela/projects/ashrae_energy_prediction/.venv/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [41]:
train_target = df_train['meter_reading']

In [30]:
train_target.max()

16.893946656813867

In [42]:
model = RandomForestRegressor(n_estimators = 100, random_state = 0, n_jobs=-1) 

In [43]:
model.fit(train_features, train_target)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
                      max_features='auto', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=-1,
                      oob_score=False, random_state=0, verbose=0,
                      warm_start=False)

In [50]:
model.predict(df_validation)

array([0.03980748, 0.24446204, 0.        , ..., 0.4928298 , 5.0811211 ,
       1.32524695])